<a href="https://colab.research.google.com/github/maddog19527/CS660_Project_3/blob/Mike's-Draft/Outlier_Detection_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, LocalOutlierFactor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import plot_tree
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.datasets import fetch_california_housing


housing=fetch_california_housing()
cal_df=pd.DataFrame(data=housing.data, columns=housing.feature_names)
cal_df['Medvalue'] = housing.target
print(cal_df.head())
print(cal_df.shape)

Z-Score Outlier Detection

In [ ]:
def ZOutliers(df):
  zdf=df.copy()
  outliers=pd.DataFrame()
  for col in zdf.columns:
    zscore=stats.zscore(zdf[col])
    zdf[f'{col}_zscore']=zscore
    outliers=pd.concat([outliers,zdf[zdf[f'{col}_zscore'].abs()>2.5]])
    outliers.drop_duplicates(inplace=True)
    z_score_columns=[col for col in outliers.columns if col.endswith('_zscore')]
    outliers=outliers.drop(columns=z_score_columns)
    outliers.to_csv('outliers.csv', index=False)
  return outliers
outliers=ZOutliers(cal_df)
print(outliers.shape)

IQR Outlier Detection

In [ ]:
def OutlierDetection(df):
  clean_df=df.copy()
  outliers=pd.DataFrame()
  for col in clean_df.columns:
    Q1=df[col].quantile(0.25)
    Q3=df[col].quantile(0.75)
    IQR=Q3-Q1
    lower=Q1-1.5*IQR
    upper=Q3+1.5*IQR
    mask=(df[col] >= lower) & (df[col]<=upper)
    outliers=pd.concat([outliers,df[~mask]])
    outliers.drop_duplicates(inplace=True)
  return outliers
cal_df=OutlierDetection(cal_df)
print(cal_df.shape)

KNN Outlier Detection

In [ ]:
def KNNOutliers(df):
  df1=df.copy()
  combinedoutliers=pd.DataFrame()
  for col in df.columns:
    k=20
    knn=KNeighborsRegressor(n_neighbors=k)
    knn.fit(df1.iloc[:,1:9], df1[col])
    distances,indeces=knn.kneighbors(df1.iloc[:,1:9])
    df1[f'outlierscore for {col}']=np.mean(distances, axis=1)
    threshold=np.percentile(np.mean(distances, axis=1), 95)
    df1[f'threshold for {col}']=threshold
    outliers=np.where(np.mean(distances, axis=1)>threshold)[0]
    outliersdoc=pd.DataFrame(outliers)
    outliersdoc.drop_duplicates(inplace=True)
    outliersdoc.columns=[col]
    combinedoutliers=pd.concat([combinedoutliers, outliersdoc], ignore_index=True)
  return combinedoutliers
outliersinfo=KNNOutliers(cal_df)
print(outliersinfo.shape)

Running the base Regression Analysis using each outlier detection program for camprison.

In [ ]:
def outliertest(df):
  KNN=KNNOutliers(df)
  Z=ZOutliers(df)
  IQR=OutlierDetection(df)
  Outliers=[KNN,Z,IQR]
  regressionresults={}
  for i, outlier in enumerate(Outliers):
    outlier_indeces=outlier.index.tolist()
    outlier_indeces=[idx for idx in outlier_indeces if idx in df.index]
    cleandf=df.drop(index=outlier_indeces)
    X=cleandf.iloc[:,1:9].values
    y=cleandf['Medvalue'].values
    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
    clf=RandomForestRegressor()
    CLFDT=DecisionTreeRegressor()
    clf.fit(x_train,y_train)
    CLFDT.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    y_predDT=CLFDT.predict(x_test)
    print(f"Mean Squared Error for Random Forest for {i + 1}: {mean_squared_error(y_test,y_pred)}")
    print(f"R2 Score for Random Forest for {i + 1}: {r2_score(y_test,y_pred)}")
    print(f"Mean Squared Error for Decision Tree for {i + 1}: {mean_squared_error(y_test,y_predDT)}")
    print(f"R2 Score for Decision Tree for {i + 1}: {r2_score(y_test,y_predDT)}")
    regressionresults[f"Outlier Method {i + 1}"] = {
        "RandomForest": {
            "Mean Squared Error": mean_squared_error(y_test, y_pred),
            "R2 Score": r2_score(y_test, y_pred)
        },
        "DecisionTree": {
            "Mean Squared Error": mean_squared_error(y_test, y_predDT),
            "R2 Score": r2_score(y_test, y_predDT)
        }
    }


outliertest(cal_df)